# HOUSİNG VERİ SETİNİN İNCELENMESİ

In [4]:
!pip install pyspark


  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488530 sha256=9765323e7c42f14aaee66818bb12ac4a4a118b2a2991172228bbdfdf0b07b89e
  Stored in directory: c:\users\meliha\appdata\local\pip\cache\wheels\92\09\11\aa01d01a7f005fda8a66ad71d2be7f8aa341bddafb27eee3c7
Successfully built pyspark


In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.mllib import linalg
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler


In [2]:
spark = SparkSession.builder \
    .appName("example-app") \
    .getOrCreate()

In [3]:
df=spark.read.csv(r"C:\Users\meliha\Spark_Veri_Analizi\housing.csv", inferSchema=True, header=True)
df.show()
#housing veri setindeki ilk 20 satır gözlemlenir. 

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [4]:
df.columns
#sütunlar incleendi

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']

In [5]:
df.printSchema()
#veri setinin ağaç yapısı oluşturuldu. ocean_proximity string değer alırken diğer bütün değişkenler double değer almaktadır.

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [6]:
df.describe().show()
#veri setinin ortalama, max, std, min değerleri incelendi. temiz bir görünüm elde edilemediğinden dolayı 
#bir alt kodda her bir featura ait değerler gözlemlendi

+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|  count|              20640|            20640|             20640|             20640|             20433|             20640|            20640|             20640|             20640|          20640|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488| 537.8705525375618|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           NULL|
| stddev|  2.0035317

In [7]:
for i in df.columns:
    df.describe(i).show()
#her ir sütunun özeti çıkarıldı mean değerleine bakıldı
#mean değerleri incelendiğinde birbirleri arasındaki orantısızlık göz önüne çıkmaktadır
#normalizasyon işlemi gerekmektedir.

+-------+-------------------+
|summary|          longitude|
+-------+-------------------+
|  count|              20640|
|   mean|-119.56970445736148|
| stddev|  2.003531723502584|
|    min|            -124.35|
|    max|            -114.31|
+-------+-------------------+

+-------+-----------------+
|summary|         latitude|
+-------+-----------------+
|  count|            20640|
|   mean| 35.6318614341087|
| stddev|2.135952397457101|
|    min|            32.54|
|    max|            41.95|
+-------+-----------------+

+-------+------------------+
|summary|housing_median_age|
+-------+------------------+
|  count|             20640|
|   mean|28.639486434108527|
| stddev| 12.58555761211163|
|    min|               1.0|
|    max|              52.0|
+-------+------------------+

+-------+------------------+
|summary|       total_rooms|
+-------+------------------+
|  count|             20640|
|   mean|2635.7630813953488|
| stddev|2181.6152515827944|
|    min|               2.0|
|    max|  

In [8]:
df.select('ocean_proximity').distinct().count()
#ocean_proximity string değer aldğından kaç tane unique değer aldığı gözlemlendi

5

In [9]:
df.select('ocean_proximity').distinct().collect()
#kategorik değişken olan ocean_proximity için 5  farklı değer gözlemlenildi

[Row(ocean_proximity='ISLAND'),
 Row(ocean_proximity='NEAR OCEAN'),
 Row(ocean_proximity='NEAR BAY'),
 Row(ocean_proximity='<1H OCEAN'),
 Row(ocean_proximity='INLAND')]

In [10]:
liste= ['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']
#veri setine ait columnlar bir listede tutuldu.

In [11]:
for column_name in liste:
    null_count = df.where(df[column_name].isNull()).count()
    if null_count > 0:
        print("{} sütunu {} tane null deger iceriyor".format(df[column_name], null_count))
    else:
        print("{} sütunu null deger icermiyor".format(df[column_name]))
#her bir featurdaki toplam null değerlrele bakıldı

Column<'longitude'> sütunu null deger icermiyor
Column<'latitude'> sütunu null deger icermiyor
Column<'housing_median_age'> sütunu null deger icermiyor
Column<'total_rooms'> sütunu null deger icermiyor
Column<'total_bedrooms'> sütunu 207 tane null deger iceriyor
Column<'population'> sütunu null deger icermiyor
Column<'households'> sütunu null deger icermiyor
Column<'median_income'> sütunu null deger icermiyor
Column<'median_house_value'> sütunu null deger icermiyor
Column<'ocean_proximity'> sütunu null deger icermiyor


In [12]:
my_df=df.dropDuplicates()
duplicate_rows = df.count() - my_df.count()
duplicate_rows
#tekrarlanan satır, veri varsa silindi

0

In [13]:
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import IndexToString , StringIndexer ,VectorIndexer, OneHotEncoder
from pyspark.ml import Pipeline

imputer = Imputer(inputCols=["total_bedrooms"], outputCols=["imputed_total_bedrooms"]).setStrategy("mean")
stringIndexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_index")
encoder = OneHotEncoder(inputCol="ocean_proximity_index", outputCol="ocean", dropLast=False)

pipeline = Pipeline(stages=[imputer, stringIndexer, encoder])
model = pipeline.fit(df)
transformed_df = model.transform(df)

transformed_df.show()
#pipe line oluşturularak boş olan sütundaki değerler mean ile dolduurldu ve string olan değişken için
#one hot encoder işlemi yapıldı. ocean_proximity deki near bay değişkeni
#bu sütunda 3. indekste bulunduğundan indeksi 3.0 dır. ocean_proximity değişkenin vektör uzayı alınarak sütun olarak eklendi

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+----------------------+---------------------+-------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|imputed_total_bedrooms|ocean_proximity_index|        ocean|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+----------------------+---------------------+-------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                 129.0|                  3.0|(5,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|                1106.0|                  3.0|(5,[3],[1.0])|
|  -122.24|   3

In [14]:
transformed_df=transformed_df.drop("total_bedrooms")
transformed_df=transformed_df.drop("ocean_proximity")
transformed_df.show()
#bu iki değer silindi, kategorik dğeşken silindi, içinde boş değer barındıran total_bedrooms değeri 
#silindi. ocean_proximity deki stirng değerler sonradan eklenen ocean_proximity_index sütununda index değeri verildiğinden
#silindi.

+---------+--------+------------------+-----------+----------+----------+-------------+------------------+----------------------+---------------------+-------------+
|longitude|latitude|housing_median_age|total_rooms|population|households|median_income|median_house_value|imputed_total_bedrooms|ocean_proximity_index|        ocean|
+---------+--------+------------------+-----------+----------+----------+-------------+------------------+----------------------+---------------------+-------------+
|  -122.23|   37.88|              41.0|      880.0|     322.0|     126.0|       8.3252|          452600.0|                 129.0|                  3.0|(5,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|    2401.0|    1138.0|       8.3014|          358500.0|                1106.0|                  3.0|(5,[3],[1.0])|
|  -122.24|   37.85|              52.0|     1467.0|     496.0|     177.0|       7.2574|          352100.0|                 190.0|                  3.0|(5,[3],[1.0])|
|  -

In [15]:
for i in transformed_df.columns:
    transformed_df.describe(i).show()
    #veri setindeki null değerler doldurulduktan sonra std, min, max vb. değerler tekrardan incelendi.

+-------+-------------------+
|summary|          longitude|
+-------+-------------------+
|  count|              20640|
|   mean|-119.56970445736148|
| stddev|  2.003531723502584|
|    min|            -124.35|
|    max|            -114.31|
+-------+-------------------+

+-------+-----------------+
|summary|         latitude|
+-------+-----------------+
|  count|            20640|
|   mean| 35.6318614341087|
| stddev|2.135952397457101|
|    min|            32.54|
|    max|            41.95|
+-------+-----------------+

+-------+------------------+
|summary|housing_median_age|
+-------+------------------+
|  count|             20640|
|   mean|28.639486434108527|
| stddev| 12.58555761211163|
|    min|               1.0|
|    max|              52.0|
+-------+------------------+

+-------+------------------+
|summary|       total_rooms|
+-------+------------------+
|  count|             20640|
|   mean|2635.7630813953488|
| stddev|2181.6152515827944|
|    min|               2.0|
|    max|  

In [16]:
liste2=['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'imputed_total_bedrooms',
 'ocean_proximity_index',
 'ocean']
#verideki feauture lar bir listede tutuldu.

In [17]:
from pyspark.ml.feature import VectorAssembler, Normalizer

assembler = VectorAssembler(inputCols=liste2, outputCol="features")

assembler_output = assembler.transform(transformed_df)
#vektörize yapıldıktan sonra normalize işlemi yapıldı

normalizer = Normalizer(inputCol="features", outputCol="Norm", p=1.0)

l1NormData = normalizer.transform(assembler_output)

l1NormData.show()
##normalizasyon işkemi uygulandı

+---------+--------+------------------+-----------+----------+----------+-------------+------------------+----------------------+---------------------+-------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|population|households|median_income|median_house_value|imputed_total_bedrooms|ocean_proximity_index|        ocean|            features|                Norm|
+---------+--------+------------------+-----------+----------+----------+-------------+------------------+----------------------+---------------------+-------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|     322.0|     126.0|       8.3252|          452600.0|                 129.0|                  3.0|(5,[3],[1.0])|[-122.23,37.88,41...|[-2.6906879807440...|
|  -122.22|   37.86|              21.0|     7099.0|    2401.0|    1138.0|       8.3014|          358500.0|                1106.0|                  3.0|(5,[3],[1.0])|[-1

In [18]:
l1NormData.columns
#normalize edilmiş olan sütunlarun vektör uzayı sütun eklenerek yukarıda gösterildi

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'imputed_total_bedrooms',
 'ocean_proximity_index',
 'ocean',
 'features',
 'Norm']

In [30]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

assembler = VectorAssembler(inputCols=['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'population',
 'households',
 'median_income',
 'imputed_total_bedrooms',
 'ocean_proximity_index',
 'ocean', 'features',
 'Norm'], outputCol='features')

train_data, test_data = l1NormData.randomSplit([0.7, 0.3], seed=42)

lr = LinearRegression(featuresCol="features", labelCol="median_house_value")
model = lr.fit(train_data.select('features', 'median_house_value'))
predictions = model.transform(test_data)

evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))
#verideki featurelar vektörize edildi, data ½70 train ve %30 test olmak üzere ayrıldı. hedef değişken median_house_value dir
#lineer regression modeli ile eğitildi, rmse (ortalama karesel hata) ölçüldü.


Root Mean Squared Error (RMSE) on test data: 0.245


In [31]:
print("Coefficients: ", model.coefficients)
print("Intercept: {:.3f}".format(model.intercept))
#modelin kat sayıları gözlemlendi. 
#modelin y ekseninde kestiği nokta hesaplandı.

Coefficients:  [0.26561543539969423,0.2703469488889847,0.002897550081074118,0.00022142214422635147,0.00010447337061745348,-0.0016731673240378277,-0.05025508409065163,1.000000552978515,0.00012292594316298865,0.027339082974535196,0.13148318839533113,-0.324980079431043,0.15423649264321665,0.21366372248886478,0.23745371368106372]
Intercept: 22.138
